# Homework: Named entity recognition

Для заданной тестовой выборки построить модель для обнаружения и классификации именованных сущностей (named entities). На базе корпуса CoNLL 2002.  

Чем больше baseline'ов вы превзойдете, тем выше ваша оценка
Метрика качества f1 (f1_macro) (чем выше, тем лучше)
 
baseline 1: 0.0604      random labels  
baseline 2: 0.3966      PoS features + logistic regression  
baseline 3: 0.7559      word2vec cbow embedding + baseline 2 + svm    

Пока мы рассмотрели только линейные модели - поэтому в примерах есть только они. Желательно при решении домашнего задания пользоваться линейными моделями. Таким образом, основные цели задания - feature engineering, hyperparam tuning & model selection.

! Your results must be reproducible. Если ваша модель - стохастическая, то вы явно должны задавать все seed и random_state в параметрах моделей  
! Вы должны использовать df_test только для измерения качества конечной обученной модели. 

bonus, think about:  
1. how can you exploit that words belong to some sentence?
2. why we selected f1 score with macro averaging as our classification quality measure? What other metrics are suitable   

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [3]:
df = pd.read_csv('ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


Each entity is a chunk.

Tags:
    - B-* for the word in the Beginning chunk
    - I-* for the word inside the chunk
    - O for the word outside any chunk
    
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon

In [4]:
# number of sentences
df.sentence_idx.max()

1500.0

In [5]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [6]:
# sentence length
#some magic with columns, indexes etc
tdf = df.set_index('sentence_idx') #use column sentence_idx as indexes
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [7]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,Thousands,O,48
1,1.0,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,of,O,48
2,1.0,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,demonstrators,O,48
3,1.0,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,have,O,48
4,1.0,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,marched,O,48


In [8]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [9]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48


In [10]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

#stratify parameter makes a split so that the proportion of values is the same as in target variable
df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [11]:
df_train.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
36858,153.0,35,that,18,camps,30,16,16,June,Soh,found,O,54
39120,259.0,7,the,28,to,32,30,5,and,was,known,O,58
53612,910.0,15,dictator,10,former,7,30,1,",",showed,the,O,50
6150,280.0,9,of,15,risk,7,9,1,",",despite,the,O,44
3771,169.0,10,Mexican,1,",",16,15,16,Saltillo,plant,Friday,B-tim,52


In [18]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict
#from glove import Corpus, Glove

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    

class GloveWrapper(TransformerMixin):
    def __init__(self, window=5, learning_rate=0.05, size=100, epochs=100, random_state=SEED, verbose=False):
        self.window_ = window
        self.learning_rate_ = learning_rate
        self.size_ = size
        self.epochs_ = epochs
        self.glove = None
        self.random_state = random_state
        self.verbose_ = verbose
    
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        corpus = Corpus()
        corpus.fit(sentences_list, window=self.window_)
        self.glove = Glove(no_components=self.size_, 
                           learning_rate=self.learning_rate_, 
                           random_state=self.random_state)
        self.glove.fit(corpus.matrix, 
                  epochs=self.epochs_, 
                  no_threads=4, verbose=self.verbose_)
        self.glove.add_dictionary(corpus.dictionary)
        return self
    
    def has(self, word):
        return word in self.glove.dictionary

    def transform(self, X):
        """
        X: list of words
        """
        if self.glove is None:
            raise Exception('model not fitted')
            
        return np.array([self.glove.word_vectors[self.glove.dictionary[w]] if w in self.glove.dictionary else np.zeros(self.size_) 
                         for w in X])

C:\Users\Anastasia\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
%%time
# we have to fit embedding models on whole dataset as they depend on word order
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm_notebook

# notice, that our dataset has window=2
sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

Wall time: 32.5 ms


In [22]:
sentences_list[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country'

In [23]:
w2v_Tcbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_Tcbow.fit(sentences_list)

In [24]:
w2v_Fcbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=False, random_state=SEED)
w2v_Fcbow.fit(sentences_list)

In [25]:
import pickle
with open('w2v_Tcbow.pickle', 'wb') as f:
    pickle.dump(w2v_Tcbow, f)

In [26]:
import pickle
with open('w2v_Fcbow.pickle', 'wb') as f:
    pickle.dump(w2v_Fcbow, f)

In [27]:
import pickle
with open('w2v_Tcbow.pickle', 'rb') as f:
    w2v_Tcbow = pickle.load(f)

In [28]:
with open('w2v_Fcbow.pickle', 'rb') as f:
    w2v_Fcbow = pickle.load(f)

In [ ]:
#w2v_gl = GloveWrapper(window=2, learning_rate=0.05, size=300, epochs=300, random_state=SEED, verbose=False)
#w2v_gl.fit(sentences_list)

In [34]:
%%time
# baseline 3
# use word2vec cbow embedding + baseline 2 + svm
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
import scipy.sparse as sp

embeding = w2v_Tcbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

C:\Users\Anastasia\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Anastasia\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Wall time: 21.6 s


In [ ]:
# my_baseline - baseline 3 + tuned parameters

In [44]:
params = {'C':[1, 10],
         'tol':[0.0001, 0.01],
          'class_weight':['balanced', None]}

model = model_selection.GridSearchCV(LinearSVC(multi_class='ovr', penalty='l2', random_state=SEED),
                                     param_grid=params,
                                    cv=3, scoring='f1_macro', n_jobs=-1, verbose=10, error_score=0)

model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed: 25.0min remaining:  5.0min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 29.8min finished


train 0.975019643538
test 0.780895356252


In [45]:
model.best_params_

{'C': 1, 'class_weight': None, 'tol': 0.01}